In [2]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, Discriminator
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

wandb.login()
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [3]:
def _distance_to_geodesic_criterion(predicted_geodesic, true_geodesic):
    # the inputs here are single samples from a geodesic; should be shape num_samples x num_dims
    # for each input point, we want the closest distance to any point on the true geodesic using the euclidean distance, torch.cdist
    D = torch.cdist(predicted_geodesic, true_geodesic)
    min_dists_to_true_geodesic = D.min(dim=1)[0]
    # we take the mean of the squared distances
    return torch.mean(min_dists_to_true_geodesic**2)
# def distance_to_geodesic_criterion(
#     predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
#     true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
#     ):
#     """
#     Mean of the squared distances from each predicted point to the closest point on the true geodesic
#     """
#     dists = []
#     for i in range(predicted_geodesic.shape[0]):
#         dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
#     dists = torch.stack(dists)
#     return dists.mean()

def distance_to_geodesic_criterion_len(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    lengths=1.,
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    dists = dists / lengths
    return dists.mean()

def distances_to_geodesic(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    return dists

In [4]:
entity = "xingzhis"
project = "dmae"
# sweep_id = 'a72qr26q'
sweep_id = 'dcxgbhjp'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [5]:
names = df[(df['loss_epoch']!='NaN')][['data_name']].values.flatten()

In [6]:
df[(df['loss_epoch']=='NaN')][['data_name']].values.flatten()

array([], dtype=object)

In [7]:

res_list = []
missing = []
failed = []

for data_name in tqdm(names):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'a72qr26q'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x
        gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed.append(data_name)
        print(e)

res_df = pd.DataFrame(res_list)


  0%|          | 0/36 [00:00<?, ?it/s]/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
 11%|█         | 4/36 [00:03<00:26,  1.20it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_15_0.1
single positional indexer is out-of-bounds


 39%|███▉      | 14/36 [00:16<00:16,  1.30it/s]

Geod not found 8     tlxawe92
10    tlxawe92
Name: run_id, dtype: object saddle_10_0.1
list index out of range


 50%|█████     | 18/36 [00:19<00:12,  1.41it/s]

...

 53%|█████▎    | 19/36 [00:20<00:10,  1.64it/s]

Geod not found 9     dp2zljfz
11    dp2zljfz
Name: run_id, dtype: object saddle_50_0.1
list index out of range


 61%|██████    | 22/36 [00:22<00:07,  1.77it/s]

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.3
single positional indexer is out-of-bounds


100%|██████████| 36/36 [00:34<00:00,  1.06it/s]


In [8]:
failed

['torus_15_0.1',
 'saddle_10_0.1',
 'saddle_15_0.3',
 'saddle_50_0.1',
 'hemisphere_15_0.3']

In [9]:

res_list = []
missing = []
failed2 = []

for data_name in tqdm(failed):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        sweep_id = '88x8glfh'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x
        gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed2.append(data_name)
        print(e)

res_df2 = pd.DataFrame(res_list)


  0%|          | 0/5 [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
 20%|██        | 1/5 [00:00<00:00,  4.71it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_15_0.1
single positional indexer is out-of-bounds


 40%|████      | 2/5 [00:00<00:01,  2.62it/s]

Geod not found Series([], Name: run_id, dtype: object) saddle_15_0.3
single positional indexer is out-of-bounds


100%|██████████| 5/5 [00:02<00:00,  1.88it/s]

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.3
single positional indexer is out-of-bounds


In [10]:
results_ours = pd.concat([res_df, res_df2], ignore_index=True)

In [11]:
results_ours.sort_values('name')

,name,length_corr,length_mse,dist2geod
21,ellipsoid_10_0.1,0.336414,7.626499,0.29572147
29,ellipsoid_15_0,0.921134,13.538965,0.022677947
28,ellipsoid_15_0.1,0.820483,18.791481,0.06582518
30,ellipsoid_15_0.3,0.576552,13.403852,0.32661584
19,ellipsoid_15_0.5,0.647232,10.757952,0.6226368
23,ellipsoid_15_0.7,0.060474,11.378612,1.1153214
27,ellipsoid_50_0.1,0.566694,24.697193,0.15313408
22,ellipsoid_5_0.1,0.544357,10.093882,0.06106465
24,ellipsoid_none_0.1,0.377259,4.839061,0.13107541
16,hemisphere_10_0.1,0.833427,2.442398,0.059005648


In [12]:
results_ours

,name,length_corr,length_mse,dist2geod
0,torus_5_0.1,0.890268,9.415816,0.05384674
1,torus_none_0.1,0.924677,4.056273,0.11373744
2,torus_15_0.7,0.558442,14.995730,0.9642704
3,torus_50_0.1,0.917187,34.002651,0.1104999
4,torus_15_0.3,0.940678,23.389124,0.22142553
5,torus_15_0.5,0.851181,18.345373,0.5162952
6,torus_10_0.1,0.906720,15.638974,0.06293617
7,saddle_15_0.1,0.824788,3.760745,0.05282901
8,saddle_15_0.7,0.344530,2.878682,1.804467
9,saddle_15_0,0.973656,5.030990,0.0126630645


In [13]:
results_ours.to_csv('geodesics_density.csv', index=False)

In [14]:
failed2

['torus_15_0.1', 'saddle_15_0.3', 'hemisphere_15_0.3']

: 